In [24]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [25]:

latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
latestdir

# dfAge = pd.read_csv(latestdir+'\\Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte.csv',delimiter=';',encoding='latin1')

dfCurAdmitted = pd.read_csv(latestdir+'\\Gennembruds_DB\\01_indlagte_pr_vaccinationsstatus.csv',delimiter=';',encoding='latin1')
df = pd.read_csv(latestdir+'\\Gennembruds_DB\\06_nyindlagte_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv',delimiter=';',encoding='latin1')

df['Nyindlagte pr. 100.000'] = pd.to_numeric(df['Nyindlagte pr. 100.000'].str.replace(',','.'))

In [26]:
# curCount.values[0]
df

,Uge,Aldersgruppe,Vaccinationsstatus,Antal nyindlagte,Befolkningstørrelse,Nyindlagte pr. 100.000
0,2020-W52,12-15,Forventet fuld effekt,0,0,0.00
1,2020-W52,12-15,Ikke vaccineret,2,283610,0.71
2,2020-W52,12-15,Påbegyndt vaccinationsforløb,0,0,0.00
3,2020-W52,16-19,Forventet fuld effekt,0,0,0.00
4,2020-W52,16-19,Ikke vaccineret,7,278165,2.52
5,2020-W52,16-19,Påbegyndt vaccinationsforløb,0,0,0.00
6,2020-W52,20-39,Forventet fuld effekt,0,0,0.00
7,2020-W52,20-39,Ikke vaccineret,99,1485292,6.67
8,2020-W52,20-39,Påbegyndt vaccinationsforløb,0,6,0.00
9,2020-W52,40-64,Forventet fuld effekt,0,0,0.00


In [27]:
# Shorten data to use
firstWeek = 20
firstIndexToUse = df[df.Uge == '2021-W20'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfVacc = dfShort[dfShort.Vaccinationsstatus == 'Forventet fuld effekt']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]

allAgeGroups = df.Aldersgruppe.unique()

fullMatVaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfVacc[(dfVacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatVaccInci[:,ageIndex] = ageInci
    fullMatVaccCount[:,ageIndex] = ageCount
            
# fig,ax1 = plt.subplots()
# ax1.plot(weekRange,ageCount)


# # list(dfVacc.groupby('Aldersgruppe'))
# thisdf = list(dfVacc.groupby('Aldersgruppe'))[1][1]

# curWeekNum = pd.to_numeric(thisdf.Uge.apply(lambda x: x[-2:]))

# thisdf
# # fig,ax1 = plt.subplots()
# # ax1.plot(curWeekNum,thisdf['Nyindlagte pr. 100.000'])

In [28]:
# Shorten data to use
firstWeek = 20
firstIndexToUse = df[df.Uge == '2021-W20'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfUnvacc = dfShort[dfShort.Vaccinationsstatus == 'Ikke vaccineret']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]

allAgeGroups = df.Aldersgruppe.unique()

fullMatUnvaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfUnvacc[(dfUnvacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatUnvaccInci[:,ageIndex] = ageInci
    fullMatUnvaccCount[:,ageIndex] = ageCount
            
# fig,ax1 = plt.subplots()
# ax1.plot(weekRange,ageCount)


# # list(dfUnvacc.groupby('Aldersgruppe'))
# thisdf = list(dfUnvacc.groupby('Aldersgruppe'))[1][1]

# curWeekNum = pd.to_numeric(thisdf.Uge.apply(lambda x: x[-2:]))

# thisdf
# # fig,ax1 = plt.subplots()
# # ax1.plot(curWeekNum,thisdf['Nyindlagte pr. 100.000'])

In [29]:
# dfVacc
# list(dfVacc.groupby('Aldersgruppe'))
# thisdf = list(dfVacc.groupby('Aldersgruppe'))[1][1]

# curWeekNum = pd.to_numeric(thisdf.Uge.apply(lambda x: x[-2:]))

# thisdf
# fig,ax1 = plt.subplots()
# ax1.plot(curWeekNum,thisdf['Nyindlagte pr. 100.000'],'*-')

In [30]:
fig,ax1 = plt.subplots(figsize=(14,8))


levelsToShow = [0,1,5,10,20,50]
hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# myColors = ((0.8, 0.0, 0.0, 1.0), (0.0, 0.8, 0.0, 1.0), (0.0, 0.0, 0.8, 1.0))
cmap = LinearSegmentedColormap.from_list('Custom', hex_colors, len(hex_colors))

g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.0f',
                vmax=30,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

plt.title('Vaccinated, hospitalizations per 100.000')
plt.tight_layout()
plt.savefig(path_figs+'HeatmapBreakthrough_Vaccinated')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
fig,ax1 = plt.subplots(figsize=(14,8))


levelsToShow = [0,1,5,10,20,50]
hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# myColors = ((0.8, 0.0, 0.0, 1.0), (0.0, 0.8, 0.0, 1.0), (0.0, 0.0, 0.8, 1.0))
cmap = LinearSegmentedColormap.from_list('Custom', hex_colors, len(hex_colors))

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.0f',
                vmax=30,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

plt.title('Unvaccinated, hospitalizations per 100.000')
plt.tight_layout()

plt.savefig(path_figs+'HeatmapBreakthrough_Unvaccinated')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [117]:

# from matplotlib.colors import ListedColormap
# cmap = ListedColormap(['red','black','yellow', 'blue'])
# # colorbar.set_ticklabels(['A', 'B', 'C', 'NA'])

# fig,ax1 = plt.subplots(figsize=(16,8))
# g = sns.heatmap(fullMatVaccInci.transpose(),
#                 ax=ax1,
#                 annot=True,
#                 fmt='1.0f',
#                 xticklabels=weekLabels,
#                 yticklabels=allAgeGroups,
#                 cmap=cmap,
#                 )
# g.invert_yaxis()
# plt.yticks(rotation=0)

In [32]:

from matplotlib.colors import LinearSegmentedColormap


# curMax = fullMatVaccInci.max()


levelsToShow = [0,1,5,10,20,50]
# boundaries = np.array([0.0,5,10,20,50]) / 50
# # boundaries = [0.0, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]  # custom boundaries
# # boundaries = np.arange(0,1,10)
# # hex_colors = sns.light_palette('navy', n_colors=len(boundaries) * 2 + 2, as_cmap=False).as_hex()
# hex_colors = sns.light_palette('navy', n_colors=len(boundaries) + 2, as_cmap=False).as_hex()
# hex_colors = [hex_colors[i] for i in range(0, len(hex_colors), 1)]
# colors=list(zip(boundaries, hex_colors))
# custom_color_map = LinearSegmentedColormap.from_list(
#     name='custom_navy',
#     colors=colors,
# )

hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) + 1, as_cmap=False).as_hex()
# myColors = ((0.8, 0.0, 0.0, 1.0), (0.0, 0.8, 0.0, 1.0), (0.0, 0.0, 0.8, 1.0))
cmap = LinearSegmentedColormap.from_list('Custom', hex_colors, len(hex_colors))


fig,ax1 = plt.subplots(figsize=(16,8))
g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.1f',
                vmax=50,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                # cbar_kws={'ticks': [0.0, 0.2, 0.4, 0.5, 0.7, 0.8, 1.0]}
                )
g.invert_yaxis()

# colorbar = ax1.collections[0].colorbar
# colorbar.set_ticks(levelsToShow)
# colorbar.set_ticklabels(['B', 'A', 'C'])
plt.yticks(rotation=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([0.5, 1.5, 2.5, 3.5, 4.5, 5.5]),
 [Text(0, 0.5, '12-15'),
  Text(0, 1.5, '16-19'),
  Text(0, 2.5, '20-39'),
  Text(0, 3.5, '40-64'),
  Text(0, 4.5, '65-79'),
  Text(0, 5.5, '80+')])

In [127]:

# curdf = df 
curdf = dfShort[(dfShort.Aldersgruppe == '80+') & (dfShort.Vaccinationsstatus == 'Ikke vaccineret')]
curdf = dfShort[(dfShort.Aldersgruppe == '80+') & (dfShort.Vaccinationsstatus == 'Forventet fuld effekt')]
curdf

curWeekNum = curdf.Uge.apply(lambda x: int(x[6:]))

curAge = '20-39'
for curAge in dfShort.Aldersgruppe.unique():
    curAgeLabel = 'Personer i aldersgruppen '+curAge
    agedf = dfShort[(dfShort.Aldersgruppe == curAge)]

    curSums = agedf.groupby('Uge').sum()

    fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)


    # ax1.plot(curWeekNum,curSums['Antal nyindlagte'])

    agedfVacc = agedf[agedf.Vaccinationsstatus == 'Forventet fuld effekt']
    agedfUnvacc = agedf[agedf.Vaccinationsstatus == 'Ikke vaccineret']
    # ax1.fill_between(curWeekNum,agedfUnvacc['Antal nyindlagte'].values,color='xkcd:light red')
    # ax1.fill_between(curWeekNum,agedfVacc['Antal nyindlagte'].values,agedfUnvacc['Antal nyindlagte'].values,color='b')

    curWeekNum = agedfVacc.Uge.apply(lambda x: int(x[6:]))

    hospUn = agedfUnvacc['Antal nyindlagte'].values
    hospVa = agedfVacc['Antal nyindlagte'].values
    hospTot = hospUn + hospVa

    # ax1.plot(curWeekNum,hospUn)
    # ax1.plot(curWeekNum,hospTot,'k')
    ax1.fill_between(curWeekNum,100*hospTot/hospTot,color='b',label='Vaccinerede')
    ax1.fill_between(curWeekNum,100*hospUn/hospTot,color='xkcd:light red',label='Uvaccinerede')
    # ax1.plot(rnTime(curWeekNum,5),rnMean(100*hospUn/hospTot,5),color='r')
    # ax1.plot(curWeekNum,hospVa/hospTot,color='b')
    # ax1.fill_between(curWeekNum,hospVa/hospTot,hospUn/hospTot,color='b')
    # ax2.fill_between(curWeekNum,agedfUnvacc['Befolkningstørrelse'].values,color='xkcd:light red')
    # ax2.fill_between(curWeekNum,agedfVacc['Befolkningstørrelse'].values,agedfUnvacc['Befolkningstørrelse'].values,color='b')

    popUn = agedfUnvacc['Befolkningstørrelse'].values
    popVa = agedfVacc['Befolkningstørrelse'].values
    popTot = popUn + popVa
    ax2.fill_between(curWeekNum,100*popTot/popTot,color='b')
    ax2.fill_between(curWeekNum,100*popUn/popTot,color='xkcd:light red')
    # ax1.plot(curWeekNum,agedfUnvacc['Antal nyindlagte'].values,color='xkcd:light red')
    # ax1.plot(curWeekNum,agedfVacc['Antal nyindlagte'].values,color='b')
    # ax1.stackplot(curWeekNum,
    #                 agedfUnvacc['Antal nyindlagte'],
    #                 agedfVacc['Antal nyindlagte'])

    # plt.plot(curWeekNum,curdf['Nyindlagte pr. 100.000'])
    # # plt.plot(curWeekNum,curdf['Befolkningstørrelse'])

    ax1.grid(axis='y')
    ax2.grid(axis='y')
    ax1.set_ylabel('Andel af alle indlæggelser [%]')
    ax2.set_ylabel('Andel af befolkningsstørrelse [%]')

    ax1.set_xlim([curWeekNum.values[0],curWeekNum.values[-1]])
    ax1.set_title(curAgeLabel)

    curMax = 100
    ax1.set_ylim(bottom=0,top=curMax)
    ax2.set_ylim(bottom=0,top=curMax)
    fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-127-c3c13671c328>:33: RuntimeWarning: invalid value encountered in true_divide
  ax1.fill_between(curWeekNum,100*hospTot/hospTot,color='b',label='Vaccinerede')
<ipython-input-127-c3c13671c328>:34: RuntimeWarning: invalid value encountered in true_divide
  ax1.fill_between(curWeekNum,100*hospUn/hospTot,color='xkcd:light red',label='Uvaccinerede')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:

# curdf = df 
curdf = shortdf[(shortdf.Aldersgruppe == '80+') & (shortdf.Vaccinationsstatus == 'Ikke vaccineret')]
curdf

plt.figure()

# plt.plot(curdf.)

,Uge,Aldersgruppe,Vaccinationsstatus,Antal nyindlagte,Befolkningstørrelse,Nyindlagte pr. 100.000
16,2020-W52,80+,Ikke vaccineret,236,271763,86.84
31,2020-W53,80+,Ikke vaccineret,244,267439,91.24
49,2021-W01,80+,Ikke vaccineret,197,254927,77.28
64,2021-W02,80+,Ikke vaccineret,107,243733,43.90
82,2021-W03,80+,Ikke vaccineret,94,233261,40.30
97,2021-W04,80+,Ikke vaccineret,58,232075,24.99
112,2021-W05,80+,Ikke vaccineret,29,230036,12.61
130,2021-W06,80+,Ikke vaccineret,25,220882,11.32
148,2021-W07,80+,Ikke vaccineret,27,188430,14.33
166,2021-W08,80+,Ikke vaccineret,16,152069,10.52
